In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
import glob
import os
import datetime
pd.set_option("max_columns", 200)



for y in ['20200512']:
    print('----------------------------------------------------------------')
    print(y)
    readPath = '\\\\192.168.10.30\\Kevin_zhenyu\\temp\\GTA\\' + y + '\\SSEL2\\STOCK\\TAQ\\***\\***'
    dataPathLs = np.array(glob.glob(readPath))
    dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs >= 600000) & (dateLs <= 700000)]
    logSH1 = []
    startTm = datetime.datetime.now()
    for i in dataPathLs:
        df = pd.read_csv(i)
        logSH1 += [df]
    logSH1 = pd.concat(logSH1).reset_index(drop=True)
    print(datetime.datetime.now() - startTm)
    logSH1 = logSH1.rename(columns={"Symbol":"StockID", "TotalVolume":"cum_volume", "TotalAmount":"cum_amount", "LastPrice":"close", 
                            "OpenPrice":"openPrice","BuyPrice01":"bid1p", "BuyPrice02":"bid2p", "BuyPrice03":"bid3p", 
                            "BuyPrice04":"bid4p", "BuyPrice05":"bid5p", "BuyVolume01":"bid1q", "BuyVolume02":"bid2q", 
                            "BuyVolume03":"bid3q", "BuyVolume04":"bid4q", "BuyVolume05":"bid5q","SellPrice01":"ask1p", 
                            "SellPrice02":"ask2p", "SellPrice03":"ask3p", "SellPrice04":"ask4p", "SellPrice05":"ask5p",
                            "SellVolume01":"ask1q", "SellVolume02":"ask2q", "SellVolume03":"ask3q", "SellVolume04":"ask4q", 
                            "SellVolume05":"ask5q"})

    
    path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdLog_SH_*'))[0]
    path2 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdLog_SZ_*'))[0]
    path3 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zt_96_04_day_96data\\mdLog_SH_*'))[0]
    
    
    startTm = datetime.datetime.now()
    logSH2 = pd.read_csv(path3,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]
    print(datetime.datetime.now() - startTm)
    logSH2["time"] = logSH2["time"].apply(lambda x: int((x.replace(':', "")).replace(".", ""))) 
    logSH1["time"] = logSH1["TradingTime"].astype(str).str[11:13] + logSH1["TradingTime"].astype(str).str[14:16] + \
    logSH1["TradingTime"].astype(str).str[17:19] + logSH1["TradingTime"].astype(str).str[20:]
    logSH1["time"] = logSH1["time"].astype(int)
    
    
    print('----------------------------------------------------------------')
    print('SH lv2 data:')
    in_dex = [16, 300, 852, 905]
    data1 = logSH2[~logSH2["StockID"].isin(in_dex) & (logSH2["time"] >= 91500000) & (logSH2["time"] <= 145700000) \
              & (logSH2["source"] == 13)]
    data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1["time"] >= 91500000) & (logSH1["time"] <= 145700000)]
    data1["cum_amount"] = data1["cum_amount"].round(2)
    data2["cum_amount"] = data2["cum_amount"].round(2)
    columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())
    print(n1)
    print(n2)
    if n1 != n2:
        sl = list(set(data1_1["StockID"].unique()) & set(data2_1["StockID"].unique()))
        data1_1 = data1_1[data1_1["StockID"].isin(sl)]
        data2_1 = data2_1[data2_1["StockID"].isin(sl)]
    
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["time_x"].count()
    n2 = test["time_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["time_y"])])
        display(len(test[np.isnan(test["time_y"])])/n1)
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(test[np.isnan(test["time_x"])])
        display(n2-n1)
        print((n2-n1)/n1)
#     del logSH2
#     del logSH1
#     del data1
#     del data2
#     del test
#     del data1_1
#     del data2_1
    
    
#     print('----------------------------------------------------------------')
#     print('SH index data:')
    
#     readPath = '\\\\192.168.10.30\\Kevin_zhenyu\\temp\\GTA\\' + y + '\\SSEL2\\INDEX\\TAQ\\***\\***'
#     dataPathLs = np.array(glob.glob(readPath))
#     dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
#     dataPathLs = dataPathLs[(dateLs == 16) | (dateLs == 300) | (dateLs == 852) | (dateLs == 905)]
#     index = []
#     for i in dataPathLs:
#         df = pd.read_csv(i)
#         index += [df]
#     index = pd.concat(index).reset_index(drop=True)
#     index = index.rename(columns={"Symbol":"StockID", "TotalVolume":"cum_volume", "TotalAmount":"cum_amount", "LastPrice":"close", 
#                             "OpenPrice":"openPrice","BuyPrice01":"bid1p", "BuyPrice02":"bid2p", "BuyPrice03":"bid3p", 
#                             "BuyPrice04":"bid4p", "BuyPrice05":"bid5p", "BuyVolume01":"bid1q", "BuyVolume02":"bid2q", 
#                             "BuyVolume03":"bid3q", "BuyVolume04":"bid4q", "BuyVolume05":"bid5q","SellPrice01":"ask1p", 
#                             "SellPrice02":"ask2p", "SellPrice03":"ask3p", "SellPrice04":"ask4p", "SellPrice05":"ask5p",
#                             "SellVolume01":"ask1q", "SellVolume02":"ask2q", "SellVolume03":"ask3q", "SellVolume04":"ask4q", 
#                             "SellVolume05":"ask5q"})
#     logSH = pd.read_csv(path1,
#                     encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
#                                               "exchange", "time", "cum_volume", "cum_amount", "close",
#                                               "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
#                                               "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
#                                               "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
#                                               "ask4q", "ask5q", "openPrice"]]
#     logSH["time"] = logSH["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
#     index["time"] = index["TradingTime"].astype(str).str[11:13] + index["TradingTime"].astype(str).str[14:16] + \
#     index["TradingTime"].astype(str).str[17:19] + index["TradingTime"].astype(str).str[20:]
#     index["time"] = index["time"].astype(int)
    
#     print(index["StockID"].unique())
#     data1 = logSH[(logSH["StockID"].isin(in_dex)) & (logSH["cum_volume"] > 0) & (logSH["time"] <= 145700000)]
#     data2 = index[(index["StockID"].isin(in_dex)) & (index["cum_volume"] > 0) & (index["time"] <= 145700000)]

#     data2["cum_volume"] = data2["cum_volume"]/100


#     columns = ["StockID", "cum_volume", "close", "openPrice"]
#     data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
#     data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

#     data2_1["close"] = data2_1["close"].round(4)

#     test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
#     n1 = test["time_x"].count()
#     n2 = test["time_y"].count()
#     len1 = len(test)
#     print(n1)
#     print(n2)
#     print(len1)
#     if n2 < len1:
#         display("test is not complete:")
#         display(test[np.isnan(test["time_y"])])
#     # 2036 unmatched for the reason like openPrice==0 when closePrice != 0
#     del index
#     del logSH
#     del data1
#     del data2
#     del test
#     del data1_1
#     del data2_1
    
#     print('----------------------------------------------------------------')
#     print('SZ lv2 data:')
    
#     readPath = '\\\\192.168.10.30\\Kevin_zhenyu\\temp\\GTA\\' + y + '\\SZSEL2\\STOCK\\TAQ\\***\\***'
#     dataPathLs = np.array(glob.glob(readPath))
#     dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
#     dataPathLs = dataPathLs[(dateLs < 4000) | (dateLs > 300000)]
#     logSZ1 = []
#     for i in dataPathLs:
#         df = pd.read_csv(i)
#         logSZ1 += [df]
#     logSZ1 = pd.concat(logSZ1).reset_index(drop=True)
#     logSZ1 = logSZ1.rename(columns={"Symbol":"StockID", "TotalVolume":"cum_volume", "TotalAmount":"cum_amount", "LastPrice":"close", 
#                             "OpenPrice":"openPrice","BuyPrice01":"bid1p", "BuyPrice02":"bid2p", "BuyPrice03":"bid3p", 
#                             "BuyPrice04":"bid4p", "BuyPrice05":"bid5p", "BuyVolume01":"bid1q", "BuyVolume02":"bid2q", 
#                             "BuyVolume03":"bid3q", "BuyVolume04":"bid4q", "BuyVolume05":"bid5q","SellPrice01":"ask1p", 
#                             "SellPrice02":"ask2p", "SellPrice03":"ask3p", "SellPrice04":"ask4p", "SellPrice05":"ask5p",
#                             "SellVolume01":"ask1q", "SellVolume02":"ask2q", "SellVolume03":"ask3q", "SellVolume04":"ask4q", 
#                             "SellVolume05":"ask5q"})
#     logSZ = pd.read_csv(path2,
#                     encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
#                                               "exchange", "time", "cum_volume", "cum_amount", "close",
#                                               "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
#                                               "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
#                                               "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
#                                               "ask4q", "ask5q", "openPrice"]]
#     logSZ["time"] = logSZ["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))  
#     logSZ1["time"] = logSZ1["TradingTime"].astype(str).str[11:13] + logSZ1["TradingTime"].astype(str).str[14:16] + \
#     logSZ1["TradingTime"].astype(str).str[17:19] + logSZ1["TradingTime"].astype(str).str[20:]
#     logSZ1["time"] = logSZ1["time"].astype(int)  
    
    
#     data1 = logSZ[(logSZ["cum_volume"] > 0) & (logSZ["time"] < 145700000) ]
#     data2 = logSZ1[(logSZ1["cum_volume"] > 0) & (logSZ1["time"] < 145700000)]

#     columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
#            "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
#            "ask4q", "ask5q", "openPrice"]
#     data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
#     data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
#     data1_1["cum_amount"] = data1_1["cum_amount"].round(2)
#     n1 = len(data1_1["StockID"].unique())
#     n2 = len(data2_1["StockID"].unique())
#     print(n1)
#     print(n2)
#     if n1 != n2:
#         sl = list(set(data1_1["StockID"].unique()) & set(data2_1["StockID"].unique()))
#         data1_1 = data1_1[data1_1["StockID"].isin(sl)]
#         data2_1 = data2_1[data2_1["StockID"].isin(sl)]
#     test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
#     n1 = test["time_x"].count()
#     n2 = test["time_y"].count()
#     len1 = len(test)
#     print(n1)
#     print(n2)
#     print(len1)
#     print("-----------------------------------------------")
#     if n2 < len1:
#         display("test is not complete:")
#         display(test[np.isnan(test["time_y"])])
#         print(len(test[np.isnan(test["time_y"])])/n1)
#         print(np.sort(test[np.isnan(test["time_y"])]["time_x"].unique()))
#         print(len(np.sort(test[np.isnan(test["time_y"])]["StockID"].unique())))
#         print(np.sort(test[np.isnan(test["time_y"])]["StockID"].unique()))
#     if (len1 == n2) & (n1 < len1):
#         display("baseline is not complete:")
#         display(test[np.isnan(test["time_x"])])
#         display(n2-n1)
#     del logSZ
#     del logSZ1
#     del data1
#     del data2
#     del test
#     del data1_1
#     del data2_1
    
    
    
    
    
#     readPath = '\\\\192.168.10.30\\Kevin_zhenyu\\temp\\GTA\\' + y + '\\SZSEL2\\STOCK\\ORDER\\***\\***'
#     dataPathLs = np.array(glob.glob(readPath))
#     dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
#     dataPathLs = dataPathLs[(dateLs < 4000) | (dateLs > 300000)]
#     OrderLogSZ1 = []
#     for i in dataPathLs:
#         df = pd.read_csv(i)
#         OrderLogSZ1 += [df]
#     OrderLogSZ1 = pd.concat(OrderLogSZ1).reset_index(drop=True)
#     OrderLogSZ1 = OrderLogSZ1.rename(columns={"OrderPrice":"Price", "OrderVolume":"OrderQty", "SecurityID":"xx", "Symbol":"SecurityID", 
#                                           "OrderCode":"Side", "RecID":"ApplSeqNum"})

#     OrderLogSZ1["Price"] = OrderLogSZ1["Price"] * 10000
#     OrderLogSZ1["Price"] = OrderLogSZ1["Price"].round(0)
#     OrderLogSZ1["TransactTime"] = OrderLogSZ1["TradingTime"].astype(str).str[11:13] + OrderLogSZ1["TradingTime"].astype(str).str[14:16] + \
#     OrderLogSZ1["TradingTime"].astype(str).str[17:19] + OrderLogSZ1["TradingTime"].astype(str).str[20:]
#     OrderLogSZ1["TransactTime"] = OrderLogSZ1["TransactTime"].astype(int)

#     path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdOrderLog_*'))[0]
#     OrderLogSZ = pd.read_csv(path1,
#                     encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
#                                                  "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
#                                                  "OrderQty"]]
#     OrderLogSZ["OrderType"] = OrderLogSZ["OrderType"].apply(lambda x: str(x))
#     OrderLogSZ1["OrderType"] = OrderLogSZ1["OrderType"].apply(lambda x: str(x))

#     sl = list(set(OrderLogSZ["SecurityID"].unique()) & set(OrderLogSZ1['SecurityID'].unique()))
#     OrderLogSZ = OrderLogSZ[OrderLogSZ["SecurityID"].isin(sl)]
#     OrderLogSZ1 = OrderLogSZ1[OrderLogSZ1["SecurityID"].isin(sl)]
#     print(len(OrderLogSZ["SecurityID"].unique()))
#     print(len(OrderLogSZ1["SecurityID"].unique()))
    
#     print('----------------------------------------------------------------')
#     print('SZ order data:')
    
#     columns = ["ApplSeqNum", "TransactTime","Side",'OrderType', 'Price', 'OrderQty', "SecurityID"]
#     ree = pd.merge(OrderLogSZ, OrderLogSZ1, on=columns, how="outer", validate='one_to_one')
#     n1 = ree["sequenceNo"].count()
#     n2 = ree["xx"].count()
#     len1 = len(ree)
#     print(n1)
#     print(n2)
#     print(len1)
#     print("-----------------------------------------------")
#     if n2 < len1:
#         display("test is not complete:")
#         display(ree[np.isnan(ree["xx"])])
#         print(len(ree[np.isnan(ree["xx"])]))
#         print(np.sort(ree[np.isnan(ree["xx"])]["TransactTime"].unique()))
#         print(len(ree[np.isnan(ree["xx"])]["SecurityID"].unique()))
#         print(ree[np.isnan(ree["xx"])]["SecurityID"].unique())
#     if (len1 == n2) & (n1 < len1):
#         display("test is complete, baseline is not complete:")
#         display(ree[np.isnan(ree["sequenceNo"])])
#         print(np.sort(ree[np.isnan(ree["sequenceNo"])]["TransactTime"].unique()))
#         print(len(ree[np.isnan(ree["sequenceNo"])]["SecurityID"].unique()))
#         print(ree[np.isnan(ree["sequenceNo"])]["SecurityID"].unique())
#         display(n2-n1)
#     del OrderLogSZ
#     del OrderLogSZ1
#     del ree
    
    
    
    
    
#     readPath = '\\\\192.168.10.30\\Kevin_zhenyu\\temp\\GTA\\' + y + '\\SSEL2\\STOCK\\TRADE\\***\\***'
#     dataPathLs = np.array(glob.glob(readPath))
#     dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
#     dataPathLs = dataPathLs[(dateLs >= 600000) & (dateLs <= 700000)]
#     SH1 = []
#     for i in dataPathLs:
#         df = pd.read_csv(i)
#         SH1 += [df]
#     SH1 = pd.concat(SH1).reset_index(drop=True)

#     SH1 = SH1.rename(columns={"SecurityID":"xx", "BuySellFlag":"TradeBSFlag", "function_code":"ExecType",
#                              "trade_price":"TradePrice", "TradeVolume":"TradeQty", "SellRecID":"OfferApplSeqNum", 
#                               "BuyRecID":"BidApplSeqNum", "TradeAmount":"TradeMoney", "RecID":"ApplSeqNum"})
#     SH1["TransactTime"] = SH1["TradingTime"].astype(str).str[11:13] + SH1["TradingTime"].astype(str).str[14:16] + \
#     SH1["TradingTime"].astype(str).str[17:19] + SH1["TradingTime"].astype(str).str[20:]
#     SH1["TransactTime"] = SH1["TransactTime"].astype(int)
#     SH1["SecurityID"] = SH1["Symbol"]
#     SH1["ExecType"] = 'F'
#     SH1["TradePrice"] = SH1["TradePrice"] * 10000
#     SH1["TradePrice"] = SH1["TradePrice"].round(0)
#     SH1["TradeMoney"] = SH1["TradeMoney"] * 10000
#     SH1["TradeMoney"] = SH1["TradeMoney"].round(0)

#     path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdTradeLog_*'))[0]
#     SH = pd.read_csv(path1,
#                     encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
#                                                  "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
#                                                  "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
#                                                  "OfferApplSeqNum"]]
#     sl = list(set(SH["SecurityID"].unique()) & set(SH1['SecurityID'].unique()))
#     SH = SH[SH["SecurityID"].isin(sl)]
#     SH1 = SH1[SH1["SecurityID"].isin(sl)]
#     print(len(SH["SecurityID"].unique()))
#     print(len(SH1["SecurityID"].unique()))

#     print(SH1.columns)
    
#     print('----------------------------------------------------------------')
#     print('SH trade data:')
    
#     SH["ExecType"] = SH["ExecType"].apply(lambda x: str(x))
#     SH1["ExecType"] = SH1["ExecType"].apply(lambda x: str(x))
#     columns = ["TransactTime", "ApplSeqNum", "SecurityID", "TradePrice", "TradeQty", "TradeMoney", "TradeBSFlag","ExecType",
#            "BidApplSeqNum", "OfferApplSeqNum"]
#     n1 = len(SH["SecurityID"].unique())
#     n2 = len(SH1["SecurityID"].unique())
#     print(n1)
#     print(n2)
#     if n1 > n2:
#         SH = SH[SH["SecurityID"].isin(SH1["SecurityID"].unique())]
#     if n2 > n1:
#         SH1 = SH1[SH1["SecurityID"].isin(SH["SecurityID"].unique())]
#     re = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
#     n1 = re["sequenceNo"].count()
#     n2 = re["xx"].count()
#     len1 = len(re)
#     print(n1)
#     print(n2)
#     print(len1)
#     print("-----------------------------------------------")
#     if n2 < len1:
#         display("test is not complete:")
#         display(re[np.isnan(re["xx"])])
#         print(len(re[np.isnan(re["xx"])]))
#         print(np.sort(re[np.isnan(re["xx"])]["TransactTime"].unique()))
#         print(len(re[np.isnan(re["xx"])]["SecurityID"].unique()))
#         print(re[np.isnan(re["xx"])]["SecurityID"].unique())
#     if (len1 == n2) & (n1 < len1):
#         display("baseline is not complete:")
#         display(re[np.isnan(re["sequenceNo"])])
#         print(np.sort(re[np.isnan(re["sequenceNo"])]["TransactTime"].unique()))
#         print(len(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique()))
#         print(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique())
#         display(n2-n1)
#     del SH
#     del SH1
#     del re
    
    
    
    
    
    
#     readPath = '\\\\192.168.10.30\\Kevin_zhenyu\\temp\\GTA\\' + y + '\\SZSEL2\\STOCK\\TRADE\\***\\***'
#     dataPathLs = np.array(glob.glob(readPath))
#     dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
#     dataPathLs = dataPathLs[(dateLs < 4000) | (dateLs > 300000)]
#     TradeLogSZ1 = []
#     for i in dataPathLs:
#         df = pd.read_csv(i)
#         TradeLogSZ1 += [df]
#     TradeLogSZ1 = pd.concat(TradeLogSZ1).reset_index(drop=True)

#     TradeLogSZ1 = TradeLogSZ1.rename(columns={"SecurityID":"xx", "function_code":"ExecType",
#                              "trade_price":"TradePrice", "TradeVolume":"TradeQty", "SellOrderID":"OfferApplSeqNum", 
#                               "BuyOrderID":"BidApplSeqNum", "RecID":"ApplSeqNum", "TradeType":"ExecType"})
#     TradeLogSZ1["TransactTime"] = TradeLogSZ1["TradingTime"].astype(str).str[11:13] + TradeLogSZ1["TradingTime"].astype(str).str[14:16] + \
#     TradeLogSZ1["TradingTime"].astype(str).str[17:19] + TradeLogSZ1["TradingTime"].astype(str).str[20:]
#     TradeLogSZ1["TransactTime"] = TradeLogSZ1["TransactTime"].astype(int)
#     TradeLogSZ1["SecurityID"] = TradeLogSZ1["Symbol"]
#     TradeLogSZ1["TradePrice"] = TradeLogSZ1["TradePrice"] * 10000
#     TradeLogSZ1["TradePrice"] = TradeLogSZ1["TradePrice"].round(0)
#     TradeLogSZ1["TradeMoney"] = TradeLogSZ1["TradePrice"] * TradeLogSZ1["TradeQty"]
#     TradeLogSZ1["TradeBSFlag"] = np.where(TradeLogSZ1["SymbolSource"] == 102, 'N', 'N')
    
#     path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdTradeLog_*'))[0]

#     TradeLogSZ = pd.read_csv(path1,
#                     encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
#                                                  "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
#                                                  "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
#                                                  "OfferApplSeqNum"]]
#     sl = list(set(TradeLogSZ["SecurityID"].unique()) & set(TradeLogSZ1['SecurityID'].unique()))
#     TradeLogSZ = TradeLogSZ[TradeLogSZ["SecurityID"].isin(sl)]
#     TradeLogSZ1 = TradeLogSZ1[TradeLogSZ1["SecurityID"].isin(sl)]
#     print(len(TradeLogSZ["SecurityID"].unique()))
#     print(len(TradeLogSZ1["SecurityID"].unique()))

#     print(TradeLogSZ1.columns)
    
    
#     print('----------------------------------------------------------------')
#     print('SZ trade data:')
    
#     TradeLogSZ["ExecType"] = TradeLogSZ["ExecType"].apply(lambda x: str(x))
#     TradeLogSZ1["ExecType"] = TradeLogSZ1["ExecType"].apply(lambda x: str(x))

#     columns = ["TransactTime","ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag","TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum","OfferApplSeqNum"]
#     re = pd.merge(TradeLogSZ, TradeLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
#     n1 = re["sequenceNo"].count()
#     n2 = re["xx"].count()
#     len1 = len(re)
#     print(n1)
#     print(n2)
#     print(len1)
#     print("-----------------------------------------------")
#     if n2 < len1:
#         display("test is not complete:")
#         display(re[np.isnan(re["xx"])])
#         print(len(re[np.isnan(re["xx"])]))
#         print(np.sort(re[np.isnan(re["xx"])]["TransactTime"].unique()))
#         print(len(re[np.isnan(re["xx"])]["SecurityID"].unique()))
#         print(re[np.isnan(re["xx"])]["SecurityID"].unique())
#     if (len1 == n2) & (n1 < len1):
#         display("baseline is not complete:")
#         display(re[np.isnan(re["sequenceNo"])])
#         print(np.sort(re[np.isnan(re["sequenceNo"])]["TransactTime"].unique()))
#         print(len(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique()))
#         print(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique())
#         display(n2-n1)
#     del TradeLogSZ
#     del TradeLogSZ1
#     del re
    



----------------------------------------------------------------
20200512
0:02:18.943844
0:00:52.928220
----------------------------------------------------------------
SH lv2 data:


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1597
1599
5227910
5212074
5227910
-----------------------------------------------


'test is not complete:'

,index_x,clockAtArrival,sequenceNo,source,StockID,exchange,time_x,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,index_y,TradingDate,TradingTime,PreClosePrice,HighPrice,LowPrice,TradeStatus,SellLevelNo,SellPrice10,SellPrice09,SellPrice08,SellPrice07,SellPrice06,BuyPrice06,BuyPrice07,BuyPrice08,BuyPrice09,BuyPrice10,BuyLevelNo,SellVolume10,SellVolume09,SellVolume08,SellVolume07,SellVolume06,BuyVolume06,BuyVolume07,BuyVolume08,BuyVolume09,BuyVolume10,TotalNo,TotalBuyOrderVolume,WtAvgBuyPrice,BondWtAvgBuyPrice,TotalSellOrderVolume,WtAvgSellPrice,BondWtAvgSellPrice,IOPV,YTM,UNIX,Market,ClosePrice,TotalBuyOrderNo01,TotalBuyOrderNo02,TotalBuyOrderNo03,TotalBuyOrderNo04,TotalBuyOrderNo05,TotalBuyOrderNo06,TotalBuyOrderNo07,TotalBuyOrderNo08,TotalBuyOrderNo09,TotalBuyOrderNo10,TotalSellOrderNo01,TotalSellOrderNo02,TotalSellOrderNo03,TotalSellOrderNo04,TotalSellOrderNo05,TotalSellOrderNo06,TotalSellOrderNo07,TotalSellOrderNo08,TotalSellOrderNo09,TotalSellOrderNo10,ETFBuyNo,ETFBuyVolume,ETFBuyAmount,ETFSellNo,ETFSellVolume,ETFSellAmount,WithdrawBuyNo,WithdrawBuyVolume,WithdrawBuyAmount,WithdrawSellNo,WithdrawSellVolume,WithdrawSellAmount,TotalBuyNo,TotalSellNo,MaxBuyDuration,MaxSellDuration,BuyOrderNo,SellOrderNo,SecurityID,time_y
262,249365,1589246700307952,1863897,13,600136,SH,92500000,8500,7.259000e+04,8.54,8.54,8.53,8.51,8.50,8.48,21100,500,8300,4000,1300,8.57,8.59,8.61,8.62,8.65,2500,2000,600,3700,3000,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
885,250041,1589246700637153,1871936,13,600996,SH,92501000,26900,1.979840e+05,7.36,7.36,7.27,7.26,7.23,7.22,37100,4500,1900,2000,1700,7.37,7.38,7.39,7.40,7.42,15500,2000,6000,6000,3300,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1054,250216,1589246700699900,1872111,13,600288,SH,92501000,5000,5.375000e+04,10.75,10.71,10.70,10.66,10.61,10.60,10500,2500,1000,75000,4000,10.75,10.81,10.85,10.90,10.91,5900,500,10000,500,1000,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1096,250262,1589246700709979,1872157,13,600663,SH,92501000,1702,1.933472e+04,11.36,11.36,11.35,11.33,11.32,11.31,1698,12800,6000,6100,17400,11.40,11.45,11.46,11.48,11.49,300,5200,4300,1500,3000,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1279,250460,1589246700815398,1872355,13,603311,SH,92501000,13500,1.615950e+05,11.97,11.97,11.96,11.92,11.91,11.87,5000,700,300,4500,400,11.98,11.99,12.00,12.03,12.05,5400,600,500,1000,3900,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

0.003029126362160022

In [3]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
import glob
import os
import datetime
pd.set_option("max_columns", 200)
y = '20200512'
path3 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zt_96_04_day_96data\\mdLog_SH_*'))[0]
    
    
startTm = datetime.datetime.now()
logSH2 = pd.read_csv(path3,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]
print(datetime.datetime.now() - startTm)
logSH2["time"] = logSH2["time"].apply(lambda x: int((x.replace(':', "")).replace(".", ""))) 
t1 = logSH2[(logSH2["StockID"] == 603320) & (logSH2["source"] == 13)][["StockID", "time", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice"]]
t1.to_csv(r"\\192.168.10.30\Kevin_zhenyu\temp\GTA 2 stocks\603320.csv", index=False)
t2 = logSH2[(logSH2["StockID"] == 600971) & (logSH2["source"] == 13)][["StockID", "time", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice"]]
t2.to_csv(r"\\192.168.10.30\Kevin_zhenyu\temp\GTA 2 stocks\600971.csv", index=False)

0:00:53.555819


In [11]:
kk = logSH2[(logSH2["StockID"] == 600971) & (logSH2["source"] == 13)][["StockID", "time", "clockAtArrival", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice"]]
kk[kk["time"] >= 103001000]

,StockID,time,clockAtArrival,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice
3249125,600971,103001000,1589250600296861,1842903,9159705.89,4.96,4.96,4.95,4.94,4.93,4.92,84000,136200,52500,80700,107000,4.97,4.98,4.99,5.00,5.01,25000,39600,60800,152501,100600,4.95
3249191,600971,103001000,1589250600489225,1844303,9166649.89,4.96,4.96,4.95,4.94,4.93,4.92,82600,136200,52500,80700,107000,4.97,4.98,4.99,5.00,5.01,25000,39600,60800,152501,100600,4.95
3251500,600971,103004000,1589250603489493,1844403,9167146.89,4.97,4.96,4.95,4.94,4.93,4.92,82600,136200,52500,80700,107000,4.97,4.98,4.99,5.00,5.01,24900,39600,60800,152501,100600,4.95
3253701,600971,103007000,1589250606679703,1844403,9167146.89,4.97,4.96,4.95,4.94,4.93,4.92,87600,136200,52500,80700,107000,4.97,4.98,4.99,5.00,5.01,24900,39600,60800,152501,100600,4.95
3255692,600971,103010000,1589250609680703,1845403,9172106.89,4.96,4.96,4.95,4.94,4.93,4.92,90400,136200,52500,80700,107000,4.97,4.98,4.99,5.00,5.01,24900,39600,60800,152901,100600,4.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11412885,600971,150505000,1589267102017784,5975504,29585340.64,4.94,4.94,4.93,4.92,4.91,4.90,80600,227400,138500,189500,116100,4.95,4.96,4.97,4.98,4.99,75300,88812,157200,57100,49600,4.95
11416924,600971,150606000,1589267162007370,5975504,29585340.64,4.94,4.94,4.93,4.92,4.91,4.90,80600,227400,138500,189500,116100,4.95,4.96,4.97,4.98,4.99,75300,88812,157200,57100,49600,4.95
11421053,600971,150706000,1589267222006410,5975504,29585340.64,4.94,4.94,4.93,4.92,4.91,4.90,80600,227400,138500,189500,116100,4.95,4.96,4.97,4.98,4.99,75300,88812,157200,57100,49600,4.95
11425172,600971,150805000,1589267282003969,5975504,29585340.64,4.94,4.94,4.93,4.92,4.91,4.90,80600,227400,138500,189500,116100,4.95,4.96,4.97,4.98,4.99,75300,88812,157200,57100,49600,4.95


In [16]:
test[np.isnan(test["time_y"]) & (test["time_x"] > 103000000)]

,index_x,clockAtArrival,sequenceNo,source,StockID,exchange,time_x,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,index_y,TradingDate,TradingTime,PreClosePrice,HighPrice,LowPrice,TradeStatus,SellLevelNo,SellPrice10,SellPrice09,SellPrice08,SellPrice07,SellPrice06,BuyPrice06,BuyPrice07,BuyPrice08,BuyPrice09,BuyPrice10,BuyLevelNo,SellVolume10,SellVolume09,SellVolume08,SellVolume07,SellVolume06,BuyVolume06,BuyVolume07,BuyVolume08,BuyVolume09,BuyVolume10,TotalNo,TotalBuyOrderVolume,WtAvgBuyPrice,BondWtAvgBuyPrice,TotalSellOrderVolume,WtAvgSellPrice,BondWtAvgSellPrice,IOPV,YTM,UNIX,Market,ClosePrice,TotalBuyOrderNo01,TotalBuyOrderNo02,TotalBuyOrderNo03,TotalBuyOrderNo04,TotalBuyOrderNo05,TotalBuyOrderNo06,TotalBuyOrderNo07,TotalBuyOrderNo08,TotalBuyOrderNo09,TotalBuyOrderNo10,TotalSellOrderNo01,TotalSellOrderNo02,TotalSellOrderNo03,TotalSellOrderNo04,TotalSellOrderNo05,TotalSellOrderNo06,TotalSellOrderNo07,TotalSellOrderNo08,TotalSellOrderNo09,TotalSellOrderNo10,ETFBuyNo,ETFBuyVolume,ETFBuyAmount,ETFSellNo,ETFSellVolume,ETFSellAmount,WithdrawBuyNo,WithdrawBuyVolume,WithdrawBuyAmount,WithdrawSellNo,WithdrawSellVolume,WithdrawSellAmount,TotalBuyNo,TotalSellNo,MaxBuyDuration,MaxSellDuration,BuyOrderNo,SellOrderNo,SecurityID,time_y
1454238,3249191,1589250600489225,44924510,13,600971,SH,103001000,1844303,9.166650e+06,4.96,4.96,4.95,4.94,4.93,4.92,82600,136200,52500,80700,107000,4.97,4.98,4.99,5.00,5.01,25000,39600,60800,152501,100600,4.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1454608,3249586,1589250601165638,44929549,13,600740,SH,103001000,4634164,2.635585e+07,5.74,5.73,5.72,5.71,5.70,5.69,89000,15700,43800,19000,34000,5.74,5.75,5.76,5.77,5.78,2600,3152,53900,89100,129300,5.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1454991,3250004,1589250601803901,44933734,13,603115,SH,103002000,852749,1.541525e+07,18.10,18.10,18.09,18.08,18.07,18.06,1700,1500,6500,5100,1000,18.12,18.13,18.14,18.15,18.16,1400,1000,700,4600,1300,18.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1455211,3250824,1589250602835460,44940079,13,600311,SH,103003000,11912790,1.860354e+07,1.56,1.56,1.55,1.54,1.53,1.52,471689,1176100,503200,801800,621000,1.57,1.58,1.59,1.60,1.61,742742,884464,1190400,1630451,3782136,1.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1456142,3252429,1589250604962697,44958420,13,601068,SH,103005000,1836749,7.367611e+06,4.01,4.01,4.00,3.99,3.98,3.97,56700,145700,241400,160100,111200,4.02,4.03,4.04,4.05,4.06,92495,37500,44100,44800,78600,4.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [20]:
display(data1_1[(data1_1["StockID"] == 600971) & (data1_1["time"] >= 102940000)][["StockID", "time", "cum_volume", "cum_amount", "close", "bid1p", "bid1q", "ask1p", "ask1q", "openPrice"]].head(20))
display(data2_1[(data2_1["StockID"] == 600971) & (data2_1["time"] >= 102940000)][["StockID", "time", "cum_volume", "cum_amount", "close", "bid1p", "bid1q", "ask1p", "ask1q", "openPrice"]].head(20))

,StockID,time,cum_volume,cum_amount,close,bid1p,bid1q,ask1p,ask1q,openPrice
1448662,600971,102944000,1840603,9148297.89,4.97,4.96,86300,4.97,25000,4.95
1449691,600971,102947000,1841803,9154249.89,4.96,4.96,85100,4.97,25000,4.95
1451058,600971,102952000,1841803,9154249.89,4.96,4.96,85100,4.97,25000,4.95
1453185,600971,102958000,1842903,9159705.89,4.96,4.96,84000,4.97,25000,4.95
1454238,600971,103001000,1844303,9166649.89,4.96,4.96,82600,4.97,25000,4.95
1455356,600971,103004000,1844403,9167146.89,4.97,4.96,82600,4.97,24900,4.95
1456522,600971,103007000,1844403,9167146.89,4.97,4.96,87600,4.97,24900,4.95
1457645,600971,103010000,1845403,9172106.89,4.96,4.96,90400,4.97,24900,4.95
1458783,600971,103013000,1845403,9172106.89,4.96,4.96,90500,4.97,24900,4.95
1460711,600971,103017000,1845403,9172106.89,4.96,4.96,91700,4.97,24900,4.95


,StockID,time,cum_volume,cum_amount,close,bid1p,bid1q,ask1p,ask1q,openPrice
3273980,600971,102944000,1840603,9148297.89,4.97,4.96,86300,4.97,25000,4.95
3273981,600971,102947000,1841803,9154249.89,4.96,4.96,85100,4.97,25000,4.95
3273982,600971,102952000,1841803,9154249.89,4.96,4.96,85100,4.97,25000,4.95
3273983,600971,102958000,1842903,9159705.89,4.96,4.96,84000,4.97,25000,4.95
3273984,600971,103004000,1844403,9167146.89,4.97,4.96,82600,4.97,24900,4.95
3273985,600971,103007000,1844403,9167146.89,4.97,4.96,87600,4.97,24900,4.95
3273986,600971,103010000,1845403,9172106.89,4.96,4.96,90400,4.97,24900,4.95
3273987,600971,103013000,1845403,9172106.89,4.96,4.96,90500,4.97,24900,4.95
3273988,600971,103017000,1845403,9172106.89,4.96,4.96,91700,4.97,24900,4.95
3273989,600971,103020000,1846603,9178058.89,4.96,4.96,90500,4.97,24900,4.95


In [5]:
display(data1_1[(data1_1["StockID"] == 603320) & (data1_1["cum_volume"] == 209000)])
display(data2_1[(data2_1["StockID"] == 603320) & (data2_1["cum_volume"] == 209000)])

,index,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice
412900,1105708,1589247900670966,19702667,13,603320,SH,94501000,209000,3360224.0,16.04,16.04,16.01,16.0,15.98,15.96,1500,4600,2000,5800,1000,16.07,16.08,16.09,16.1,16.11,7300,4000,2000,5600,8200,15.99


,index,StockID,TradingDate,TradingTime,PreClosePrice,openPrice,HighPrice,LowPrice,close,TradeStatus,SellLevelNo,SellPrice10,SellPrice09,SellPrice08,SellPrice07,SellPrice06,ask5p,ask4p,ask3p,ask2p,ask1p,bid1p,bid2p,bid3p,bid4p,bid5p,BuyPrice06,BuyPrice07,BuyPrice08,BuyPrice09,BuyPrice10,BuyLevelNo,SellVolume10,SellVolume09,SellVolume08,SellVolume07,SellVolume06,ask5q,ask4q,ask3q,ask2q,ask1q,bid1q,bid2q,bid3q,bid4q,bid5q,BuyVolume06,BuyVolume07,BuyVolume08,BuyVolume09,BuyVolume10,TotalNo,cum_volume,cum_amount,TotalBuyOrderVolume,WtAvgBuyPrice,BondWtAvgBuyPrice,TotalSellOrderVolume,WtAvgSellPrice,BondWtAvgSellPrice,IOPV,YTM,UNIX,Market,ClosePrice,TotalBuyOrderNo01,TotalBuyOrderNo02,TotalBuyOrderNo03,TotalBuyOrderNo04,TotalBuyOrderNo05,TotalBuyOrderNo06,TotalBuyOrderNo07,TotalBuyOrderNo08,TotalBuyOrderNo09,TotalBuyOrderNo10,TotalSellOrderNo01,TotalSellOrderNo02,TotalSellOrderNo03,TotalSellOrderNo04,TotalSellOrderNo05,TotalSellOrderNo06,TotalSellOrderNo07,TotalSellOrderNo08,TotalSellOrderNo09,TotalSellOrderNo10,ETFBuyNo,ETFBuyVolume,ETFBuyAmount,ETFSellNo,ETFSellVolume,ETFSellAmount,WithdrawBuyNo,WithdrawBuyVolume,WithdrawBuyAmount,WithdrawSellNo,WithdrawSellVolume,WithdrawSellAmount,TotalBuyNo,TotalSellNo,MaxBuyDuration,MaxSellDuration,BuyOrderNo,SellOrderNo,SecurityID,time
